<a href="https://colab.research.google.com/github/stabti/data_ethics_epf/blob/main/Data_ethics_Biases_in_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Author : Ikram Chraibi Kaadoud

Teacher (year 2025) : Sonia Tabti

This notebook will help understand the concept of Data Ethic through the following examples



# Use case 1: Toxicity in comments

At the end of 2017, the [Civil Comments](https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d) platform shut down and released their ~2 million public comments in a lasting open archive.
(If you cant open the medium article, see the "Saying goodbye to Civil Comments.pdf" file)
Jigsaw sponsored this effort and helped to comprehensively annotate the data.  In 2019, Kaggle held the [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview) competition so that data scientists worldwide could work together to investigate ways to mitigate bias.

The dataset contains thousands of comments, where each comment is labeled as either "toxic" or "not toxic".

### A reminder

**The tutorial below tackles the question of racial bias that a Machine Learning model can develop from comments.
The comments studied come from a real dataset and in no way reflect the ideology of the trainer or the school, nor any value judgment whatsoever with regard to a community or individual.**

**First run the code bellow to set the environment.**


In [ ]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
print("Environment settled successfully !\n")

Environment settled successfully !



### Dataset loading and preparation

**Run the code** bellow to import the needed library and load the dataset.
Feel free to display the dataset and explore it.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("data.csv", on_bad_lines='skip')

data.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10


In [ ]:
data.describe()

,id,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,parent_id,article_id,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
count,9.090200e+04,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,21687.00000,21687.000000,21687.000000,...,4.983000e+04,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000
mean,3.741905e+06,0.430254,0.020783,0.091598,0.054786,0.381360,0.025486,0.01059,0.003179,0.001616,...,3.683063e+06,281545.713736,0.258938,0.043365,0.106741,2.334360,0.559207,0.023667,1.535489,36.632560
std,2.450000e+06,0.406086,0.043101,0.182411,0.147375,0.386545,0.109004,0.08227,0.051270,0.024006,...,2.458295e+06,104028.940513,0.994396,0.240804,0.453165,4.587831,1.735908,0.106960,18.199186,147.288485
min,5.985600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,2.210600e+05,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,7.921310e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,7.825040e+05,159937.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,5.232225e+06,0.500120,0.000000,0.000000,0.000000,0.200000,0.000000,0.00000,0.000000,0.000000,...,5.208236e+06,332720.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6.000000
75%,5.785647e+06,0.810345,0.026316,0.116667,0.021739,0.785714,0.000000,0.00000,0.000000,0.000000,...,5.769276e+06,367055.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,58.000000
max,6.333872e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,6.333696e+06,399523.000000,58.000000,8.000000,16.000000,224.000000,96.000000,1.000000,1838.000000,4936.000000


**Create two variables:**
- "comments": that contains all the comments (one column contains all) that should be classified
- "target": that contains the result of the classification model (toxic or not toxic). Look at the target column, you may need to set a treshold

**Split** your dataset into train, test datasets. Use the previous variables that you created

To process text with a Machine Learning algorithm you should transform the text into numerical vectors.

**Use the following import** to transform the comments (of both the train and test datasets) into vectors:

- from sklearn.feature_extraction.text import CountVectorizer

**Congrats ! The Data successfully loaded !**

You are ready for the next step.

**Run the following code** to visualize examples of a toxic comment and a non toxic one. Feel free to explore the dataset

In [ ]:
# Preview the dataset
print("Sample toxic comment:", comments_train.iloc[22])
print("Sample not-toxic comment:", comments_train.iloc[17])

In [ ]:
data.head()

### Classifier implementation

Implement a classifier that predict the toxicity of a comment. Print the accuracy of your model

LogisticRegression model is advised but you can use any classifier that you want.

Compute typical performance metrics for classification (accuracy, F1 score for example)

Implement a fonction "classify_string" that take any comment as input and predict - using your model - the toxicity of the comment.
The comment is probably some text, dont forget to transform it into a numerical shape.

It is time to try your code ! <br>
Try to pass through the model the following comments: <br>

"I love looking at the sky dreaming about python code" <br>
"Love is stupid"<br>

Explore other examples from your own and from the dataset. Try to see what is the toxicity evaluation based on.


### To make a model more ethical, dig into the model you should !

You implemented the model that make the classification. You can visualize the comment and the toxicity.
Now, it is time to dig into the model and the data to : <br>
- 1) to explain a prediction
- 2) to see how can you act on the model or the data to make it more ethical.

In the cell below, I created a dataset that has two columns :
- one with all the sorted words (see the attribute vocabulary_ of your Coutnvectorizer instance)
- one with the coefficient of the features in the decision function of the classifier model (see [classifier.coef_ of LogisticRegression] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

Display the 10 words with the biggest coefficient

In [ ]:
# coefficients = pd.DataFrame({"word": sorted(list(vectorizer.vocabulary_.keys())), "coeff": classifier.coef_[0]})

### A reminder

**The tutorial below tackles the question of racial bias that a Machine Learning model can develop from comments.
The comments studied come from a real dataset and in no way reflect the ideology of the trainer or the school, nor any value judgment whatsoever with regard to a community or individual.**

Now let us take a closer look at how the model classifies comments.

Classify the following comments and visualize the coefficient of the words :
- I have a christian friend
- I have a muslim friend
- I have a white friend
- I have a black friend

Feel free to try out more comments, to see how the model classifies them.


### Question:
Do you see any signs of potential bias in the model?
How did the model classify "I have a christian friend" and "I have a muslim friend"?
How did it classify "I have a white friend" and "I have a black friend"?
Once you have an answer, write your comment in the next section

Write your text here :


# Measuring fairness of a Machine Learning Model

4 Fairness Metrics:

#### **1.Demographic parity / statistical parity**
Demographic parity says the model is fair if the composition of people who are selected by the model matches the group membership percentages of the applicants.

<u>Example:</u> A nonprofit is organizing an international conference, and 20,000 people have signed up to attend. The organizers write a ML model to select 100 attendees who could potentially give interesting talks at the conference. Since 50% of the attendees will be women (10,000 out of 20,000), they design the model so that 50% of the selected speaker candidates are women.

#### **2. Equal opportunity**
Equal opportunity fairness ensures that the proportion of people who should be selected by the model ("positives") that are correctly selected by the model is the same for each group. We refer to this proportion as the true positive rate (TPR) or sensitivity of the model.

<u>Example:</u> A doctor uses a tool to identify patients in need of extra care, who could be at risk for developing serious medical conditions. (This tool is used only to supplement the doctor's practice, as a second opinion.) It is designed to have a high TPR that is equal for each demographic group.

#### **3. Equal accuracy**
Alternatively, we could check that the model has equal accuracy for each group. That is, the percentage of correct classifications (people who should be denied and are denied, and people who should be approved who are approved) should be the same for each group. If the model is 98% accurate for individuals in one group, it should be 98% accurate for other groups.

<u>Example:</u> A bank uses a model to approve people for a loan. The model is designed to be equally accurate for each demographic group: this way, the bank avoids approving people who should be rejected (which would be financially damaging for both the applicant and the bank) and avoid rejecting people who should be approved (which would be a failed opportunity for the applicant and reduce the bank's revenue).

#### **4. Group unaware / "Fairness through unawareness"**
Group unaware fairness removes all group membership information from the dataset. For instance, we can remove gender data to try to make the model fair to different gender groups. Similarly, we can remove information about race or age.

<u>Example:</u> One difficulty of applying this approach in practice is that one has to be careful to identify and remove proxies for the group membership data. For instance, in cities that are racially segregated, zip code is a strong proxy for race. That is, when the race data is removed, the zip code data should also be removed, or else the ML application may still be able to infer an individual's race from the data. Additionally, group unaware fairness is unlikely to be a good solution for historical bias.
!

## # Use case 2: Is there any discrimination ?

The [Dataset](https://www.kaggle.com/datasets/hjmjerry/gender-discrimination) is from a Kaggle competition.

### Context and Content
A few years ago, the United States District Court of Houston had a case that arises under Title VII of the Civil Rights Act of 1964, 42 U.S.C. 200e et seq. The plaintiffs in this case were all female doctors at Houston College of Medicine who claimed that the College has engaged in a pattern and practice of discrimination against women in giving promotions and setting salaries. The Lead plaintiff in this action, a pediatrician and an assistant professor, was denied for promotion at the College. The plaintiffs had presented a set of data to show that female faculty at the school were less likely to be full professors, more likely to be assistant professors, and earn less money than men, on average.

### Dataset description

1 Dept:
    1=Biochemistry/Molecular Biology
    2=Physiology
    3=Genetics
    4=Pediatrics
    5=Medicine
    6=Surgery

2 Gender: 1=Male, 0=Female

3 Clin: 1=Primarily clinical emphasis, 0=Primarily research emphasis

4 Cert: 1=Board certified, 0=not certified

5 Prate Publication rate (# publications on cv)/(# years between CV date and MD date)

6 Exper: # years since obtaining MD

7 Rank: 1=Assistant, 2=Associate, 3=Full professor (a proxy for productivity)

8 Sal94: Salary in academic year 1994

9 Sal95: Salary after increment to 1994

According to your understanding, what is the y column (that the AI model should predict)?
Remember that you want to study if your dataset is indeed biased regarding the female that are full professor.

Write your answer bellow:


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("Kaggle_Gender_biases_Lawsuit.csv")

data.head()

Display the number of female doctors and male doctors that are present in the dataset. How much of them are assistant and professors ?

Let us transform the dataset, and especially the "Rank" column :<br>
    - We group the 1=Assistant and 2=Associate in the same category, designed by 0 <br>
    - we keep the category "3=Full Professor " alone. We design it by 1<br>

Run the code bellow for this.

In [ ]:
new_data = data.copy()
new_data['Rank'] = new_data['Rank'].replace(2,0)
new_data['Rank'] = new_data['Rank'].replace(1,0)
new_data['Rank'] = new_data['Rank'].replace(3,1)
new_data.head()

This transform the dataset in order to make it easier to analyze if the dataset is biased towards female or not.

Split your dataset into Training (80%) and testing (20%) datasets.


Implement a [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and train it.
Visualize it (see [tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) ).
Explore the proposed visualisation according to the depth of the decisionTreeClassifier

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Train a model and make predictions


There are  106 female doctors. Among them  90  are assistant and  16  are professors. <br>
There are  155 male doctors. Among them  86  are assistant and  69  are professors <br>
As a reminder:
 - Gender: 1=Male, 0=Female
 - Rank : 0= Not professor, 1= Professor

Plot the confusion matrix of the model.


What kind of analysis can you make ?

Write your answer bellow:

##  Measuring fairness of a machine learning model

We work with a synthetic dataset of information submitted by credit card applicants.

To load and preview the data, run the next code cell. When the code finishes running, you should see a message saying the data was successfully loaded, along with a preview of the first five rows of the data.

The dataset contains, for each applicant:
- income (in the `Income` column),
- the number of children (in the `Num_Children` column),
- whether the applicant owns a car (in the `Own_Car` column, the value is `1` if the applicant owns a car, and is else `0`), and
- whether the applicant owns a home (in the `Own_Housing` column, the value is `1` if the applicant owns a home, and is else `0`)

When evaluating fairness, we'll check how the model performs for users in different groups, as identified by the `Group` column:
- The `Group` column breaks the users into two groups (where each group corresponds to either `0` or `1`).  
- For instance, you can think of the column as breaking the users into two different races, ethnicities, or gender groupings.  If the column breaks users into different ethnicities, `0` could correspond to a non-Hispanic user, while `1` corresponds to a Hispanic user.


Run the next code cell without changes to train a simple model to approve or deny individuals for a credit card.  The output shows the performance of the model.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data, separate features from target
data = pd.read_csv("data_credit.csv")
data.head()

In [ ]:
X = data.drop(["Target"], axis=1)
y = data["Target"]

# Break into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Preview the data
print("Data successfully loaded!\n")
X_train.head()

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Train a model and make predictions
model_baseline = tree.DecisionTreeClassifier(random_state=0, max_depth=3)
model_baseline.fit(X_train, y_train)
preds_baseline = model_baseline.predict(X_test)

# Function to plot confusion matrix
def plot_confusion_matrix(estimator, X, y_true, y_pred, display_labels=["Deny", "Approve"],
                          include_values=True, xticks_rotation='horizontal', values_format='',
                          normalize=None, cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
    return cm, disp.plot(include_values=include_values, cmap=cmap, xticks_rotation=xticks_rotation,
                     values_format=values_format)

# Function to evaluate the fairness of the model
def get_stats(X, y, model, group_one, preds):

    print("X_test.columns : ", X_test.columns)
    print("y[0:5] : ", y[0:5])
    print("group_one[0:5] : ", group_one[0:5])
    print("preds[0:5] : ", preds[0:5])

    y_zero, preds_zero, X_zero = y[group_one==False], preds[group_one==False], X[group_one==False]
    y_one, preds_one, X_one = y[group_one], preds[group_one], X[group_one]

    print("Total approvals:", preds.sum())
    print("Group A:", preds_zero.sum(), "({}% of approvals)".format(round(preds_zero.sum()/sum(preds)*100, 2)))
    print("Group B:", preds_one.sum(), "({}% of approvals)".format(round(preds_one.sum()/sum(preds)*100, 2)))

    print("\nOverall accuracy: {}%".format(round((preds==y).sum()/len(y)*100, 2)))
    print("Group A: {}%".format(round((preds_zero==y_zero).sum()/len(y_zero)*100, 2)))
    print("Group B: {}%".format(round((preds_one==y_one).sum()/len(y_one)*100, 2)))

    cm_zero, disp_zero = plot_confusion_matrix(model, X_zero, y_zero, preds_zero)
    disp_zero.ax_.set_title("Group A")
    cm_one, disp_one = plot_confusion_matrix(model, X_one, y_one, preds_one)
    disp_one.ax_.set_title("Group B")

    print("\nSensitivity / True positive rate:")
    print("Group A: {}%".format(round(cm_zero[1,1] / cm_zero[1].sum()*100, 2)))
    print("Group B: {}%".format(round(cm_one[1,1] / cm_one[1].sum()*100, 2)))

# Evaluate the model
get_stats(X_test, y_test, model_baseline, X_test["Group"]==1, preds_baseline)

The confusion matrices above show how the model performs on some test data. We also print additional information (calculated from the confusion matrices) to assess fairness of the model. For instance,
- The model approved 38246 people for a credit card. Of these individuals, 8028 belonged to Group A, and 30218 belonged to Group B.
- The model is 94.56% accurate for Group A, and 95.02% accurate for Group B. These percentages can be calculated directly from the confusion matrix; for instance, for Group A, the accuracy is (39723+7528)/(39723+500+2219+7528).
- The true positive rate (TPR) for Group A is 77.23%, and the TPR for Group B is 98.03%. These percentages can be calculated directly from the confusion matrix; for instance, for Group A, the TPR is 7528/(7528+2219).

# 1) Varieties of fairness

Consider three different types of fairness covered in the tutorial:
- **Demographic parity**: Which group has an unfair advantage, with more representation in the group of approved applicants?  (Roughly 50% of applicants are from Group A, and 50% of applicants are from Group B.)
- **Equal accuracy**: Which group has an unfair advantage, where applicants are more likely to be correctly classified?
- **Equal opportunity**:  Which group has an unfair advantage, with a higher true positive rate?

In [ ]:
def visualize_model(model, feature_names, class_names=["Deny", "Approve"], impurity=False):
    display(feature_names)
    plot_list = tree.plot_tree(model, feature_names=feature_names, class_names=class_names, impurity=impurity)
    [process_plot_item(item) for item in plot_list]

def process_plot_item(item):
    split_string = item.get_text().split("\n")
    if split_string[0].startswith("samples"):
        item.set_text(split_string[-1])
    else:
        item.set_text(split_string[0])

plt.figure(figsize=(20, 6))
plot_list = visualize_model(model_baseline, feature_names=X_train.columns)

In [ ]:
# Create new dataset with gender removed
X_train_unaware = X_train.drop(["Group"],axis=1)
X_test_unaware = X_test.drop(["Group"],axis=1)

# Train new model on new dataset
model_unaware = tree.DecisionTreeClassifier(random_state=0, max_depth=3)
model_unaware.fit(X_train_unaware, y_train)

# Evaluate the model
preds_unaware = model_unaware.predict(X_test_unaware)
get_stats(X_test_unaware, y_test, model_unaware, X_test["Group"]==1, preds_unaware)

How does this model compare to the first model you trained, when you consider demographic parity, equal accuracy, and equal opportunity?

In [ ]:
Solution: When we consider demographic parity, the new model is still biased in favor of Group B, but is now a bit more fair than the original model. But now, if you consider either equal accuracy or equal opportunity, the model is biased in favor of Group A! It's also important to note that the overall accuracy of the model has dropped -- for each group, the model is making slightly less accurate decisions.

You decide to train a third potential model, this time with the goal of having each group have even representation in the group of approved applicants. (This is an implementation of group thresholds, which you can optionally read more about here.)

Run the next code cell without changes to evaluate this new model.

In [ ]:
# Change the value of zero_threshold to hit the objective
zero_threshold = 0.11
one_threshold = 0.99

# Evaluate the model
test_probs = model_unaware.predict_proba(X_test_unaware)[:,1]
preds_approval = (((test_probs>zero_threshold)*1)*[X_test["Group"]==0] + ((test_probs>one_threshold)*1)*[X_test["Group"]==1])[0]
get_stats(X_test, y_test, model_unaware, X_test["Group"]==1, preds_approval)

How does this final model compare to the previous models, when you consider demographic parity, equal accuracy, and equal opportunity?

In [ ]:
Solution: This model acheives nearly equal representation in the pool of approved applicants from each group -- if demographic parity is what we care about, then this model is much more fair than the first two models. Accuracy is roughly the same for each group, but there is a substantial drop in overall accuracy for each group. If we examine the model for equal opportunity fairness, the model is biased in favor of Group A: all individuals from Group A who should be approved are approved, whereas only 63% of individuals from Group B who should be approved are approved. (This is similar to the dynamic in the first model, with the favored group switched -- that is, in the first model, nearly all individuals from Group B who should be approved were approved by the model.

# Conclusion

This is only short exercises to explore different types of fairness, and to illustrate the tradeoff that can occur when you optimize for one type of fairness over another.  We have focused on model training here, but in practice, to really mitigate bias, or to make ML systems fair, we need to take a close look at every step in the process, from data collection to releasing a final product to users.

This lesson was a hands-on introduction to the topic, and you can continue your learning by reading blog posts from the [Partnership on AI](https://www.partnershiponai.org/research-lander/) or by following conferences like the [ACM Conference on Fairness, Accountability, and Transparency (ACM FAccT)](https://facctconference.org/).

This course was inspired from:
- https://www.kaggle.com/var0101/introduction-to-ai-ethics
- https://www.kaggle.com/code/nathanlauga/ethics-and-ai-how-to-prevent-bias-on-ml
- https://blog.research.google/2016/10/equality-of-opportunity-in-machine.html
- https://pair-code.github.io/what-if-tool/ai-fairness.html
